JUŻ WIEM  
Dwa podjeścia pierwsze prostsze Lepsze dlza SARMIX. Przekształcenie data frama jednowwymarowo tworząc 32 unikatowe kolumny i w każdą wpisać wartośc sprzedaży. każday oddzielny rząd ma inny dzień nie da się jedk tego zastosować dla Pytorcha  
Drugie wielowymiarowe, również 32 kolumny, ale nie wpisuje wartości num_sold a 1 wtedy gdy dana kombinacja występuje, dzięki czemu mam odzielną kolumnę z targetem dla Pytorcha najelepsze podjeście. Łatwiejsze w ponieważ nie będzie trzeba póżniej dekodować danych względem ID do przesłania

In [15]:
import os
import numpy as np
import pandas as pd
from pandas import IndexSlice  # For slicing by level

from datetime import datetime, timedelta,date
from numba import jit

# 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set_palette('icefire_r', 2)
import plotly.express as px
import plotly.io as pio
from plotly.io import show
import plotly.graph_objs as go

import warnings

%matplotlib inline
#Przetwarzanie

# Baza danych gównych do nauki modelu
train = pd.read_csv("train.csv") 
# Baza danych Titanica do sprawdzenia modelu
test = pd.read_csv("test.csv")

In [17]:
train['date'] = pd.to_datetime(train['date']).dt.to_period('D')
test['date'] = pd.to_datetime(test['date']

In [18]:
multi_index = pd.MultiIndex.from_frame(train[['date', 'id']], names=['date', 'id'])  # Select 'date' and 'id' columns explicitly
train.set_index(multi_index, inplace=True)
train.head(20)

id        date    country         store  \
date       id                                            
2017-01-01 0    0  2017-01-01  Argentina  Kaggle Learn   
           1    1  2017-01-01  Argentina  Kaggle Learn   
           2    2  2017-01-01  Argentina  Kaggle Learn   
           3    3  2017-01-01  Argentina  Kaggle Learn   
           4    4  2017-01-01  Argentina  Kaggle Learn   
           5    5  2017-01-01  Argentina  Kaggle Store   
           6    6  2017-01-01  Argentina  Kaggle Store   
           7    7  2017-01-01  Argentina  Kaggle Store   
           8    8  2017-01-01  Argentina  Kaggle Store   
           9    9  2017-01-01  Argentina  Kaggle Store   
           10  10  2017-01-01  Argentina     Kagglazon   
           11  11  2017-01-01  Argentina     Kagglazon   
           12  12  2017-01-01  Argentina     Kagglazon   
           13  13  2017-01-01  Argentina     Kagglazon   
           14  14  2017-01-01  Argentina     Kagglazon   
           15  15  2017-01-01     Canada  Kaggle Learn   
           16  16  2017-01-01     Canada  Kaggle Learn   
           17  17  2017-01-01     Canada  Kaggle Learn   
           18  18  2017-01-01     Canada  Kaggle Learn   
           19  19  2017-01-01     Canada  Kaggle Learn   

                                                      product  num_sold  
date       id                                                            
2017-01-01 0                Using LLMs to Improve Your Coding        63  
           1                    Using LLMs to Train More LLMs        66  
           2   Using LLMs to Win Friends and Influence People         9  
           3       Using LLMs to Win More Kaggle Competitions        59  
           4                       Using LLMs to Write Better        49  
           5                Using LLMs to Improve Your Coding        88  
           6                    Using LLMs to Train More LLMs        98  
           7   Using LLMs to Win Friends and Influence People        14  
           8       Using LLMs to Win More Kaggle Competitions        83  
           9                       Using LLMs to Write Better        69  
           10               Using LLMs to Improve Your Coding       340  
           11                   Using LLMs to Train More LLMs       371  
           12  Using LLMs to Win Friends and Influence People        53  
           13      Using LLMs to Win More Kaggle Competitions       364  
           14                      Using LLMs to Write Better       285  
           15               Using LLMs to Improve Your Coding       202  
           16                   Using LLMs to Train More LLMs       199  
           17  Using LLMs to Win Friends and Influence People        31  
           18      Using LLMs to Win More Kaggle Competitions       202  
           19                      Using LLMs to Write Better       138

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 136950 entries, (Period('2017-01-01', 'D'), 0) to (Period('2021-12-31', 'D'), 136949)
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype    
---  ------    --------------   -----    
 0   id        136950 non-null  int64    
 1   date      136950 non-null  period[D]
 2   country   136950 non-null  object   
 3   store     136950 non-null  object   
 4   product   136950 non-null  object   
 5   num_sold  136950 non-null  int64    
dtypes: int64(2), object(3), period[D](1)
memory usage: 12.2+ MB


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
Enc_col = ['country', 'store', 'product' ]

# Fit the encoder and create the mapping for each column separately
categories = {}
for col in Enc_col:
    for df in [train, test]:
        df[col + '_Enc'] = enc.fit_transform(df[col].values.reshape(-1, 1))
    categories[col] = enc.categories_[0]  # Store categories for the current column

# Print the mapping
for col in Enc_col:
    print(f"Kolumna: {col}")
    for i, category in enumerate(categories[col]):
        print(f"  Wartość zakodowana: {i} -> Wartość początkowa: {category}")

In [ ]:
categorical_features = ['id','country', 'store', 'product' ]
for df in [train, test]:
     df.drop(columns=categorical_features, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

train.head()
scaler= MinMaxScaler()
# -------------------------------------------------------------
train_num_sol_reshaped = train['num_sold'].values.reshape(-1, 1)  # Reshape to 2D
scaler.fit(train_num_sol_reshaped)
train['num_sold_scaled'] = scaler.transform(train_num_sol_reshaped)
#----------------------------------------------------------------
train.drop('num_sold', axis=1, inplace=True)

In [ ]:
--

In [ ]:
# print(pd.isnull(train).sum())

In [ ]:
# print(pd.isnull(test).sum())

In [ ]:
# cols= ['country', 'store', 'product' ]
# for col in cols:
#   df_grouped = train.groupby(col)[col].count()
#   df_grouped = df_grouped / train[col].count() * 100
#   print(f"Unique value counts and percentages for column {col}:")
#   print(df_grouped)

In [ ]:
# cols= ['country', 'store', 'product' ]
# for col in cols:
#   df_grouped = test.groupby(col)[col].count()
#   df_grouped = df_grouped / test[col].count() * 100
#   print(f"Unique value counts and percentages for column {col}:")
#   print(df_grouped)

In [ ]:
train.info()

In [ ]:
train.index

In [ ]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['day'] = pd.to_datetime(train['date']).dt.day
test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month
test['day'] = pd.to_datetime(test['date']).dt.day

In [ ]:
train = train.set_index('date') 
test = test.set_index('date')

#train.index = train.index.to_period(freq='D')

In [ ]:
train.index

In [ ]:
# fig = sns.lineplot(train, x=train.index.to_timestamp() , y='num_sold', hue=train['country']=='Spain')
# fig.show()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
Enc_col = ['country', 'store', 'product' ]

# Fit the encoder and create the mapping for each column separately
categories = {}
for col in Enc_col:
    for df in [train, test]:
        df[col + '_Enc'] = enc.fit_transform(df[col].values.reshape(-1, 1))
    categories[col] = enc.categories_[0]  # Store categories for the current column

# Print the mapping
for col in Enc_col:
    print(f"Kolumna: {col}")
    for i, category in enumerate(categories[col]):
        print(f"  Wartość zakodowana: {i} -> Wartość początkowa: {category}")

In [ ]:
train[['country_Enc', 'store_Enc', 'product_Enc']] = train[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
test[['country_Enc', 'store_Enc', 'product_Enc']] = test[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
train['num_sold'] = train['num_sold'].astype(np.int16)

In [ ]:
categorical_features = ['id','country', 'store', 'product' ]
for df in [train, test]:
     df.drop(columns=categorical_features, inplace=True)

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

train.head()
scaler= MinMaxScaler()
# -------------------------------------------------------------
train_num_sol_reshaped = train['num_sold'].values.reshape(-1, 1)  # Reshape to 2D
scaler.fit(train_num_sol_reshaped)
train['num_sold_scaled'] = scaler.transform(train_num_sol_reshaped)
#----------------------------------------------------------------
train.drop('num_sold', axis=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

Spain = train[train['country_Enc'] == 4]
train_Spain, val_Spain = train_test_split(Spain, test_size=0.25, shuffle=False)

# Create a figure with subplots
fig = go.Figure(layout=dict(xaxis_title="Date", yaxis_title="Sales"))

# Plot train_Spain data in first subplot
trace1 = go.Scatter(
    x=train_Spain.index(),
    y=train_Spain['num_sold_scaled'],
    name="Train",
    mode="markers",
    marker=dict(color="blue", size=2),
)

# Plot val_Spain data in second subplot
trace2 = go.Scatter(
    x=val_Spain.index(),
    y=val_Spain['num_sold_scaled'],
    name="Validation",
    mode="markers",
    marker=dict(color="red", size=2)
)
# trace3 = go.Scatter(x=Spain.index.to_timestamp(), y=Spain['num_sold_scaled'], name="spline",
#                     text=["tweak line smoothness<br>with 'smoothing' in line object"],
#                     hoverinfo='text+name',
#                     line_shape='spline', 
#                     line=dict(smoothing=1.3)  # Adjust this value to control smoothness
# )

# Add traces to the figure
fig.add_trace(trace1)
fig.add_trace(trace2)
# fig.add_trace(trace3)

# Display the chart
fig.show()

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess
from statsmodels.tsa.api import SARIMAX


# Assuming 'train' is your DataFrame with 'num_sold_scaled' as the target variable
det_proc = DeterministicProcess(train_Spain.index, constant=False, fourier=3 )
det_proc.in_sample()

train_with_exo = pd.concat([train_Spain['num_sold_scaled'], det_proc.in_sample()], axis=1)
model = SARIMAX(train_with_exo['num_sold_scaled'],
                     order=(1, 1, 1),  # Adjust order parameters as needed
                     seasonal_order=(1, 1, 1, 12),  # Adjust seasonal order as needed
                     exog=train_with_exo[['sin(1,7)', 'cos(1,7)',	'sin(2,7)',	'cos(2,7)',	'sin(3,7)',	'cos(3,7)']])

SARIMAX_fit = model.fit()

In [ ]:
print(SARIMAX_fit.summary())

In [ ]:
val_Spain.info()

In [ ]:
forecast = SARIMAX_fit.forecast(len(val_Spain), exog=det_proc.out_of_sample(len(val_Spain)))
forecast.info()

In [ ]:
----

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam  # Import Adam optimizer

from tqdm import tqdm
torch.set_num_threads(12)


In [ ]:
features = ['year', 'month', 'day', "country_Enc", "store_Enc", "product_Enc"]
target   = 'num_sold_scaled'

# Przygotuj dane dla modelu
X_train = train[features]
y_train = train[target]

X_test = test[features]

# Konwertuj dane na tensory PyTorch
X_tensor_train = torch.from_numpy(X_train.to_numpy()).float()
y_tensor_train = torch.from_numpy(y_train.to_numpy()).float().reshape(-1, 1)

# Konwertuj dane na tensory PyTorch
X_tensor_test = torch.from_numpy(X_test.to_numpy()).float()

class MyDataset(torch.utils.data.Dataset):
  def __init__(self, X_tensor, y_tensor):
    if y_tensor is not None:
      # Handle missing labels (e.g., assign a specific value)
      self.y_tensor = [label if label is not None else -1 for label in y_tensor]
    self.X_tensor = X_tensor
    self.y_tensor = y_tensor

  def __getitem__(self, idx):
    X = self.X_tensor[idx]
    y = self.y_tensor[idx]
    return X, y

  def __len__(self):
    return len(self.X_tensor)

train_dataset = MyDataset(X_tensor_train, y_tensor_train)
test_dataset = MyDataset(X_tensor_test, None) 

train_size = int(0.20 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Create a DataLoader for batch training
batch_size = 2048 # Adjust batch size as needed
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class LSTMModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim):
    super(LSTMModel, self).__init__()
    self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=0.2)  # Add dropout for regularization
    self.fc = nn.Linear(hidden_dim, output_dim)
    self.silu = nn.SiLU()  # Add SiLU activation layer

  def forward(self, x):
    # Pass the input through LSTM layers
    x, (hidden, cell) = self.lstm(x)
    # Use the output from the last hidden layer
    x = hidden[-1]
    # Apply SiLU activation before linear layer
    x = self.silu(x)
    # Apply linear layer for prediction
    x = self.fc(x)
    return x

# Hyperparameters (adjust as needed)
input_dim = len(features)  # Number of features
hidden_dim =256
num_layers = 4
output_dim = 1  # Assuming predicting single value (num_sold)

model = LSTMModel(input_dim, hidden_dim, output_dim).float()
criterion = nn.HuberLoss()
optimizer = torch.optim.NAdam(model.parameters(), lr=0.005)  # Set learning rate

# Training loop
num_epochs = 36
best_loss = float('inf')
print(model)
loss_history = []  # Initialize a list to store loss values

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    pbar = tqdm(train_loader)  # Wrap train_loader with tqdm
    for batch_x, batch_y in pbar:
        optimizer.zero_grad()
        for x, y in zip(batch_x, batch_y):
            x = x.unsqueeze(0)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()

        # Update progress bar
        pbar.set_description(f"Epoch {epoch+1}/{num_epochs} Loss: {loss.item():.2f}")

        # Append loss to history
        loss_history.append(loss.item())

    # Plot loss history after each epoch
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history, label='Training Loss')
    plt.title(f"Training Loss (Epochs: {num_epochs})")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.legend()
    plt.show()